In [6]:
import pandas as pd
import numpy as np
import glob
import gc
import os
import math
import keras
from keras.layers import CuDNNLSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.callbacks import LambdaCallback
from keras.models import Model
from keras import regularizers
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.mobilenet_v2 import MobileNetV2

gc.enable()

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
from modules import clr, LRFinder
from modules.utils import DataGenerator

In [9]:
import tensorflow as tf
import random

def seed_everything(seed=2019):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                                  inter_op_parallelism_threads=1)
    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    
seed_everything()


In [10]:
train = pd.read_csv('data/train.csv')

# load data

In [11]:
print(train.head())

    itemid                                              title  Category  \
0   307504               nyx sex bomb pallete natural palette         0   
1   461203  etude house precious mineral any cushion pearl...         1   
2  3592295                           milani rose powder blush         2   
3  4460167                etude house baby sweet sugar powder         3   
4  5853995       bedak revlon color stay aqua mineral make up         3   

                                          image_path  
0  beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg  
1  beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg  
2  beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg  
3  beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg  
4  beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg  


In [12]:
mobile_class = np.arange(31, 58)

In [13]:
batchsize = 32
height = 299
width = 299
seed = 2019
path = ""

In [14]:
indexes = np.load('np_array/trn_index_m_fold.npy')
val_index = indexes[0]
trn_index = indexes[1]
for i in [3,2,4]:
    trn_index = np.append(trn_index, indexes[i])

In [15]:
def simple_FC_layer(cv_input, classes):
    cv_input_dense = Flatten()(cv_input.output)
    main_ = Dropout(0.5)(cv_input_dense)
    main_ = Dense(300)(main_)
    output = Dense(len(classes), activation='softmax')(main_)
    model = Model(inputs=cv_input.input, outputs=output)
    return model

In [16]:
img_paths_trn = list(train.iloc[trn_index, 3])
labels_trn = train.iloc[trn_index, 2]
labels_trn = np.array(labels_trn) - mobile_class[0]
labels_trn = keras.utils.to_categorical(labels_trn, num_classes=len(mobile_class))

img_paths_val = list(train.iloc[val_index, 3])
labels_val = train.iloc[val_index, 2]
labels_val = np.array(labels_val) - mobile_class[0]
labels_val = keras.utils.to_categorical(labels_val, num_classes=len(mobile_class))

params = {'dim': (299, 299),
          'batch_size': 32,
          'n_channels': 3,
          'shuffle': True,
          "path": path
         }

training_generator = DataGenerator(img_paths_trn, labels_trn, mobile_class, **params)
testing_generator = DataGenerator(img_paths_val, labels_val, mobile_class, **params)

In [17]:
input_tensor=Input(shape=(height, width, 3))
backbone_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)
model = simple_FC_layer(backbone_model, mobile_class)

In [18]:
model.load_weights('weights/mobile_0')

In [19]:
class TestGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, batch_size=1, dim=(299, 299), n_channels=3,
                 shuffle=True, path=''):
        'Initialization'
        self.path = path
        self.dim = dim
        self.batch_size = batch_size
        self.image_paths = image_paths
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            # Store sample
            # X[i,] = np.load('data/' + ID + '.npy')
            path_ = self.image_paths[idx]
            image = cv2.imread(self.path+path_)
            if image.size != (299, 299, 3):
                image = cv2.resize(image, (299, 299))
            X[i,] = image/255
        
        return X

In [20]:
output = model.predict_generator(testing_generator, verbose=1)
np.save('cv_predictions/cv_mobile_0.npy', output)
test = pd.read_csv('data/test.csv')
img_paths = np.array(test.iloc[:76545, 2])
test_generator = TestGenerator(img_paths, shuffle=False, batch_size=64)
output = model.predict_generator(test_generator, verbose=True)
new_img_paths = img_paths[img_paths.shape[0]-img_paths.shape[0]%64:, ]
new_test_generator = TestGenerator(new_img_paths, shuffle=False, batch_size=1)
output_temp = model.predict_generator(new_test_generator, verbose=True)
output = np.append(output, output_temp, axis=0)
np.save('cv_predictions/cv_mobile_0_test.npy', output)
model_embed = Model(inputs = model.input, outputs=model.layers[-2].output)
output = model_embed.predict_generator(testing_generator, verbose=True)
np.save('cv_predictions/cv_mobile_0_embed.npy', output)
output = model_embed.predict_generator(test_generator, verbose=True)
output_temp = model_embed.predict_generator(new_test_generator, verbose=True)
output = np.append(output, output_temp, axis=0)
np.save('cv_predictions/cv_mobile_0_test_embed.npy', output)

1/1 [==============================] - 0s 38ms/step
